### Import packages


In [1]:
import sys
import gc
from client import client
import torch
from server import server_multinomial_bitflip, server_multinomial_genrr, server_ell2
from data_generator import create_power_law, data_generator
import time
import numpy as np
from datetime import datetime

### Change settings

In [2]:
d = 3                       # data dimension; choose from {3,4,5}
n_bin = 4                   # fixed at 4 in the paper
privacy_level = 1           # privacy level \alpha: choose from {0.5, 1, 2}
sample_size   = 6000        
n_permutation = 999         # fixed at 999 in the paper
priv_mech  = 'bitflip' #choose among 'bitflip', 'genrr', 'lapu', 'disclapu'
statistic  = 'elltwo' #choose among 'chi', 'projchi', 'elltwo'. chi requires 1-dimensional multinomial data.
n_test        = 2000        
test_start    = 1
significance_level = 0.05
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(torch.get_num_threads())
###################################################################

12


### Create data generator, client, and server instances

In [3]:
data_gen = data_generator() #create data generator
LDPclient = client() #create the client, which privatizes the data

method_name = priv_mech + statistic

server_private_vec = {
    "elltwo":server_ell2(privacy_level),
    "chi":server_multinomial_genrr(privacy_level),
    "projchi":server_multinomial_bitflip(privacy_level)
    }
server_private = server_private_vec[statistic] #create the server, which conducts the test

### Run the simulations #############################

In [4]:
print(f"{method_name}, alpha={privacy_level}, sample size={sample_size}")
print("#########################################")
p_value_vec = np.zeros([n_test, 1])
statistic_vec = np.zeros([n_test, 1])
t = time.time()

for i in range(n_test):
    test_num = i + test_start
    t_start_i = time.time()
    torch.manual_seed(test_num)
    copula_mean = torch.zeros(d).to(device)

    copula_sigma_1 = (0.5 * torch.ones(d,d) + 0.5 * torch.eye(d)).to(device)
    copula_sigma_2 = copula_sigma_1.mul(5)

    data_y_priv = LDPclient.release_private_conti(
            priv_mech,
            data_gen.generate_copula_gaussian_data(sample_size, copula_mean, copula_sigma_1),
            privacy_level,
            n_bin,
            device
        )

    data_z_priv = LDPclient.release_private_conti(
            priv_mech,
            data_gen.generate_copula_gaussian_data(sample_size, copula_mean, copula_sigma_2),
            privacy_level,
            n_bin,
            device
        )

    server_private.load_private_data_multinomial(
        data_y_priv,
        data_z_priv,
        LDPclient.alphabet_size_binned,
        device,
        device
    )
    time_now = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')

    p_value_vec[i], statistic_vec[i] = server_private.release_p_value_permutation(n_permutation)
    t_end_i = time.time() - t_start_i
    print(f"pval: {p_value_vec[i]} -- {test_num}th test, time elapsed {t_end_i} -- emperical power so far (from test_start): {(p_value_vec[0:(i+1)] < significance_level).mean()}")

genrrelltwo, alpha=1, sample size=1000
#########################################
pval: [0.094] -- 1th test, time elapsed 0.9477024078369141 -- emperical power so far (from test_start): 0.0
pval: [0.85600001] -- 2th test, time elapsed 0.9870755672454834 -- emperical power so far (from test_start): 0.0
pval: [0.177] -- 3th test, time elapsed 1.042029619216919 -- emperical power so far (from test_start): 0.0
pval: [0.92400002] -- 4th test, time elapsed 1.0669326782226562 -- emperical power so far (from test_start): 0.0
pval: [0.17900001] -- 5th test, time elapsed 0.9474873542785645 -- emperical power so far (from test_start): 0.0
pval: [0.44100001] -- 6th test, time elapsed 1.0135388374328613 -- emperical power so far (from test_start): 0.0
pval: [0.24699999] -- 7th test, time elapsed 1.007143259048462 -- emperical power so far (from test_start): 0.0
pval: [0.35800001] -- 8th test, time elapsed 1.0061941146850586 -- emperical power so far (from test_start): 0.0
pval: [0.052] -- 9th test, 

KeyboardInterrupt: 